In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input
from tensorflow.keras.applications import ResNet50, ResNet50V2, Xception
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, AveragePooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

2.4.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/w


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive/PlateDataset/')
!unzip Dataset.zip

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
EPOCHS = 60
BATCH_SIZE= 60
n_classes = 3

In [ ]:
DATA_PATH = '/content/drive/MyDrive/PlateDataset/Dataset/'

In [ ]:
def recall_m(y_true, y_pred):
    K = tf.keras.backend
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    K = tf.keras.backend
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    K = tf.keras.backend
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
train_datagen_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                      shear_range=0.25,
                                      width_shift_range=0.25,
                                      height_shift_range=0.25,
                                      zoom_range=0.2,
                                      rotation_range=40,
                                      validation_split=0.2)

train_generator_aug  = train_datagen_aug.flow_from_directory(DATA_PATH,
                                                              target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                              batch_size=BATCH_SIZE,
                                                              class_mode='categorical',
                                                              subset='training')

validation_generator  = validation_datagen.flow_from_directory(DATA_PATH,
                                                              target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                              batch_size=BATCH_SIZE,
                                                              class_mode='categorical',
                                                              subset='validation')

Found 2183 images belonging to 3 classes.
Found 544 images belonging to 3 classes.


In [ ]:
def build_model():
  baseModel = Xception(
        include_top=False,
        weights="imagenet",
        input_tensor=Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        pooling='avg'
    )
    
  # for layer in baseModel.layers:
  #     layer.trainable = False
  # for layer in baseModel.layers[-40:]:
  #     layer.trainable = True

  headModel = baseModel.output
  headModel = Dropout(0.2)(headModel)
  headModel = Dense(1024, activation="relu")(headModel)
  headModel = Dropout(0.2)(headModel)
  headModel = Dense(1024, activation="relu")(headModel)
  headModel = Dropout(0.2)(headModel)
  headModel = Dense(512, activation="relu")(headModel)
  headModel = Dropout(0.2)(headModel)
  headModel = Dense(512, activation="relu")(headModel)
  headModel = Dropout(0.2)(headModel)
  headModel = Dense(n_classes, activation="softmax")(headModel)
  
  model = Model(inputs=baseModel.input, outputs=headModel)

  return model

In [ ]:
loss = CategoricalCrossentropy()
optimizer = SGD(lr=1e-2, momentum=0.9, decay=1e-2/40)
model = build_model_fine_tune()
model.compile(
    loss=loss, 
    optimizer=optimizer, 
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

cb_early_stopper = EarlyStopping(monitor = 'val_f1_m', patience = 20)
cb_checkpointer = ModelCheckpoint(filepath = '/w/best_weight', monitor = 'val_f1_m', save_best_only = True, mode = 'max')

history = model.fit(
    train_generator_aug, 
    validation_data=validation_generator, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    callbacks=[cb_checkpointer],
)

Epoch 1/80
35/35 [==============================] - 51s 1s/step - loss: 0.8897 - accuracy: 0.6191 - f1_m: 0.4051 - precision_m: 0.5409 - recall_m: 0.3813 - val_loss: 1.4230 - val_accuracy: 0.6893 - val_f1_m: 0.6858 - val_precision_m: 0.6858 - val_recall_m: 0.6858
INFO:tensorflow:Assets written to: /w/best_weight/assets
Epoch 2/80
35/35 [==============================] - 47s 1s/step - loss: 0.5709 - accuracy: 0.7046 - f1_m: 0.7290 - precision_m: 0.8463 - recall_m: 0.6414 - val_loss: 1.5679 - val_accuracy: 0.6912 - val_f1_m: 0.6887 - val_precision_m: 0.6917 - val_recall_m: 0.6858
INFO:tensorflow:Assets written to: /w/best_weight/assets
Epoch 3/80
35/35 [==============================] - 47s 1s/step - loss: 0.4349 - accuracy: 0.8635 - f1_m: 0.7794 - precision_m: 0.9391 - recall_m: 0.6722 - val_loss: 1.0034 - val_accuracy: 0.7261 - val_f1_m: 0.7105 - val_precision_m: 0.7276 - val_recall_m: 0.6944
INFO:tensorflow:Assets written to: /w/best_weight/assets
Epoch 4/80
35/35 [===================

In [ ]:
# best: 
# Epoch 55/60
# 35/35 [==============================] - 35s 996ms/step - loss: 0.0077 - accuracy: 0.9980 - val_loss: 0.3016 - val_accuracy: 0.9632

In [ ]:
!ls /w/best_weight

assets	saved_model.pb	variables


In [ ]:
best_model = tf.keras.models.load_model('/w/best_weight', custom_objects={'f1_m':f1_m, 'precision_m':precision_m, 'recall_m':recall_m})
best_model.compile(
    loss=loss, 
    optimizer='adam', 
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

In [ ]:
best_model.evaluate(validation_generator, batch_size=batch_size)

9/9 [==============================] - 8s 707ms/step - loss: 0.1821 - accuracy: 0.9729 - f1_m: 0.9732 - precision_m: 0.9732 - recall_m: 0.9732


[0.19300441443920135, 0.966911792755127, 0.96875, 0.96875, 0.96875]

In [ ]:
best_model.save("/w/modelv3.h5")

In [ ]:
download("/w/modelv3.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

test_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
  )

Y_pred = best_model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['0', '1', '2']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Found 544 images belonging to 3 classes.
Confusion Matrix
[[371   4   0]
 [  9  44   3]
 [  0   0 113]]
Classification Report
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       375
           1       0.92      0.79      0.85        56
           2       0.97      1.00      0.99       113

    accuracy                           0.97       544
   macro avg       0.96      0.93      0.94       544
weighted avg       0.97      0.97      0.97       544

